In [4]:
import re
import pandas as pd
#zaman ve tarih işlemleri için
from datetime import datetime

# Bu class dan abstract oluşturulduğunda ilk istenilen kullanılacak log dosyası ve log için gerekli olan regex formatıdır
class LogParse:
    def __init__(self, filepath):
        self.filepath = filepath
        self.pattern = re.compile(
            r'(?P<ip>\d+\.\d+\.\d+\.\d+).*\[(?P<time>.*?)\] "(?P<method>\w+) (?P<url>\S+) .*" (?P<status>\d{3}) (?P<size>\d+|-)'
        )

# Bu fonksiyonda try except ile dosya bununamazsa, bunu uyarı olarak vererek program patlamasına ve soru yaklamak için kullanıldı
# Fonksiyon normalde bir tablo (DataFrame) döndürür ve Dosya yoksa bile boş tablo döndürmek, kodun çökmesini engeller.
    
    def parse(self):
        try:
            with open (self.filepath, "r") as file:
                lines = file.readlines()
        except FileNotFoundError:
            print (f" Log dosyası bulunamadı: {self.filepath}")
            return pd.DataFrame()
            
# Boş bir liste oluşturup her satır için regex eşleştirmesi yapar,match nesnesi varsa eşleşti demektir ve listeye eklenir.
# size da byte cinsinden ,Sayıya çevrilemeyenleri NaN yapar,Eksik değerleri 0 ile doldurur ve sonuçları tam sayıya çevirir
# pd.DataFrame() ile boş bir pandas.DataFrame oluşturulur.bir şey yanlış gittiğinde  döndürülür.
# return df daha önce oluşturulmuş df adlı tabloyu döndürür.
        parsed = []
        for line in lines:
            match = self.pattern.match(line)
            if match :
                parsed.append(match.groupdict())
                
        if not parsed:
            print("Uygun formatta log satırı bulunamadı")
            return pd.DataFrame()

        df = pd.DataFrame(parsed)
        df['status'] = df['status'].astype(int)
        df['size'] = pd.to_numeric(df['size'], errors = 'coerce').fillna(0).astype(int)
        
        try:
            df['datetime'] = pd.to_datetime(df['time'], format = '%d/%b/%Y:%H:%M:%S %z')
            
        except ValueError:
            print("Tarih formatı çözümlenmedi ")
            return pd.DataFrame()
        return df


# IP'lerin dakikalık bazda istek sayısı hesaplayabilmek için "minute" hassasiyetini belirtir ve floor() sadece bu seviyenin altındaki birimleri sıfırlar.
# IP ve dakika bazında gruplayarak kaç istek atıldığını sayar.
#threshold değerinden fazla istek atanları filtreler.
class IDSRuleEngine:
    def __init__(self, df):
        self.df = df
    def rule_high_request_rate(self, threshold= 10):
        print(" 1 dakikada 10'dan fazla istek atan IP'ler:")
        self.df['minute'] = self.df['datetime'].dt.floor('min')
        request_counts =self.df.groupby(['ip', 'minute']).size()
        suspicious = request_counts[request_counts > threshold]
        if suspicious.empty:
            print("Hiçbir IP eşiği aşmadı") 

        else:
            print(suspicious)

# isin() birden fazla değeri tek seferde kontrol eder.
# IP sütununu bir satır aşağı kaydır ve bir önceki satırdaki IP ile karşılaştırarak hatalı girişle aynı IP'den mi olduğuna bakar
# cunsum grup numarası oluştururur aynı ıp art arda geliyorsa aynı grup numarası verir
# streak satırında aynı gruptakı içerikleri 0 dan başlayarak sayar
# str.startswith her bir string değerin belirli '/admin', '/login', '/wp-login' gibi başlayıp başlamadığını kontrol eder
    def rule_failed_logins(self):
        print(" üst üste 3 hatalı giriş")
        failures = self.df[self.df['status'].isin([401,403])].copy()
        if failures.empty: 
           print("hiç hatalı giriş bulunmadı")
           return

        failures['group'] = (failures['ip'] != failures['ip'].shift()).cumsum()
        failures['streak'] = failures.groupby('group').cumcount()
        flagged = failures[failures['streak'] >= 2]
        for __, row in flagged.iterrows():
            print(f"IP {row['ip']} art arda 3 hatalı giriş yaptı. Saat: {row['datetime']}")

    def rule_sensitive_pages(self):
        print("\n Hassas Sayfalara Erişim Tespit Edildi:")
        critical_pages = ['/admin', '/login', '/wp-login']
        suspicious_access = self.df[self.df['url'].str.startswith(tuple(critical_pages))]
        


        if suspicious_access.empty:
            print("Hassas sayfalara erişim gözlemlenmedi.")
        else:
            print(suspicious_access[['ip', 'url', 'datetime']])


class IntrusionDetectionSystem:
    def __init__(self, log_path):
        self.log_path = log_path

    def run(self):
        parser = LogParse(self.log_path)
        df = parser.parse()

        if df.empty:
            print(" Analiz edilecek veri bulunamadı.")
            return

        engine = IDSRuleEngine(df)
        engine.rule_high_request_rate()
        engine.rule_failed_logins()
        engine.rule_sensitive_pages()


if __name__ == "__main__":
    ids = IntrusionDetectionSystem(r"C:\Users\MONSTER\Documents\tei_try\ids_access.log")
    ids.run()
        
    

          



 1 dakikada 10'dan fazla istek atan IP'ler:
ip             minute                   
192.168.1.100  2025-08-17 10:01:00+00:00    15
dtype: int64
 üst üste 3 hatalı giriş
IP 192.168.1.100 art arda 3 hatalı giriş yaptı. Saat: 2025-08-17 10:01:24+00:00
IP 192.168.1.100 art arda 3 hatalı giriş yaptı. Saat: 2025-08-17 10:01:25+00:00
IP 192.168.1.100 art arda 3 hatalı giriş yaptı. Saat: 2025-08-17 10:01:26+00:00
IP 192.168.1.100 art arda 3 hatalı giriş yaptı. Saat: 2025-08-17 10:01:27+00:00
IP 192.168.1.100 art arda 3 hatalı giriş yaptı. Saat: 2025-08-17 10:01:28+00:00
IP 192.168.1.100 art arda 3 hatalı giriş yaptı. Saat: 2025-08-17 10:01:29+00:00
IP 192.168.1.100 art arda 3 hatalı giriş yaptı. Saat: 2025-08-17 10:01:30+00:00
IP 192.168.1.100 art arda 3 hatalı giriş yaptı. Saat: 2025-08-17 10:01:31+00:00
IP 192.168.1.100 art arda 3 hatalı giriş yaptı. Saat: 2025-08-17 10:01:32+00:00
IP 192.168.1.100 art arda 3 hatalı giriş yaptı. Saat: 2025-08-17 10:01:33+00:00

 Hassas Sayfalara Erişim Tesp